<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-3/blob/main/RNN_with_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import tensorflow as tf
import keras as ks
import random
import csv

from tensorflow.keras.layers import Dense, Input,InputLayer,Flatten,Activation,LSTM,SimpleRNN,GRU,TimeDistributed,Embedding
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Concatenate, AdditiveAttention

In [ ]:
random.seed(hash("seriously you compete with me") % 2**32 - 1)
np.random.seed(hash("i am mohammed safi") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)

In [ ]:
from tensorflow.python.keras import backend as K
K._get_available_gpus()

['/device:GPU:0']

In [ ]:
!nvidia-smi

Tue Apr 26 05:48:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    76W / 149W |    145MiB / 11441MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb
import wandb

from wandb.keras import WandbCallback

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# English as source language and Telugu as target language
source_language='en'
target_language='te'

In [ ]:
def load_data():
    path = r"/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/te/lexicons/"

    train_file_path = path + "te.translit.sampled.train.tsv"
    val_file_path = path + "te.translit.sampled.dev.tsv"

    

    path = r"/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/te/lexicons/"

    train_file_path = path + "te.translit.sampled.train.tsv"
    val_file_path = path + "te.translit.sampled.dev.tsv"

    train_tsv_file = open(train_file_path, "r", encoding="utf-8")
    train_dataset = csv.reader(train_tsv_file, delimiter="\t")

    val_tsv_file = open(val_file_path, "r", encoding="utf-8")
    val_dataset = csv.reader(val_tsv_file, delimiter="\t")

    return train_dataset, val_dataset


In [ ]:
def prepare_data():

    train_dataset, val_dataset = load_data()
    
    input = []
    target = []
    #print(english)
    for i in train_dataset:
        #print(i) 
        target.append(i[0])
        #print(i[1])
        input.append(i[1])
    #print(english)
    target = np.array(target)
    input = np.array(input)

    # Validation data
    val_input = []
    val_target = []

    for i in val_dataset:
        val_target.append(i[0])
        val_input.append(i[1])

    val_target = np.array(val_target)
    val_input = np.array(val_input)

    for i in range(len(target)):
        target[i] = "\t" + target[i] + "\n"
    
    for i in range(len(val_target)):
        val_target[i] = "\t" + val_target[i] + "\n"

    return input, target, val_input, val_target



In [ ]:
def getTokens(input, target, val_input, val_target):
    # Getting input and target language characters

    # Training set
    input_tokens = set()
    target_tokens = set()

    for word in input:
        for char in word:
            if char not in input_tokens:
                input_tokens.add(char)

    for word in target:
        for char in word:
            if char not in target_tokens:
                target_tokens.add(char)

    # Validation set
    val_input_tokens = set()
    val_target_tokens = set()

    for word in val_input:
        for char in word:
            if char not in val_input_tokens:
                val_input_tokens.add(char)

    for word in val_target:
        for char in word:
            if char not in val_target_tokens:
                val_target_tokens.add(char)

    input_tokens = sorted(list(input_tokens))
    target_tokens = sorted(list(target_tokens))
    
    return input_tokens, target_tokens, val_input_tokens, val_target_tokens


In [ ]:
input, target, val_input, val_target = prepare_data()
input_tokens, target_tokens, val_input_tokens, val_target_tokens = getTokens(input, target, val_input, val_target)


In [ ]:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(txt) for txt in input])
max_decoder_seq_length = max([len(txt) for txt in target])

input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

print("Number of data points:", len(input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of data points: 58550
Number of unique input tokens: 26
Number of unique output tokens: 65
Max sequence length for inputs: 25
Max sequence length for outputs: 20


ENCODER AND DECODER INPUTS

In [ ]:
def one_hot_encoding(input, target, val_input, val_target, input_tokens, target_tokens):

    input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])

    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

    
    encoder_input_data = np.zeros(
        (len(input), max_encoder_seq_length), dtype = "float32" )
    decoder_input_data = np.zeros(
        (len(input), max_decoder_seq_length), dtype = "float32" )
    decoder_target_data = np.zeros(
        (len(input), max_decoder_seq_length, num_decoder_tokens), dtype = "float32" )
    
    for i, (inp, tar) in enumerate(zip(input, target)):
        for t, char in enumerate(inp):
            encoder_input_data[i,t] = input_token_index[char]

        for t, char in enumerate(tar):
            decoder_input_data[i,t] = target_token_index[char]
            if t>0:
                decoder_target_data[i,t-1,target_token_index[char]] = 1.0

    

    encoder_val_input_data = np.zeros(
        (len(val_input), max_encoder_seq_length), dtype = "float32" )
    decoder_val_input_data = np.zeros(
        (len(val_input), max_decoder_seq_length), dtype = "float32" )
    decoder_val_target_data = np.zeros(
        (len(val_input), max_decoder_seq_length, num_decoder_tokens), dtype = "float32" )

    for i, (inp, tar) in enumerate(zip(val_input, val_target)):
        for t, char in enumerate(inp):
            encoder_val_input_data[i,t] = input_token_index[char]

        for t, char in enumerate(tar):
            decoder_val_input_data[i,t] = target_token_index[char]
            if t>0:
                decoder_target_data[i,t-1,target_token_index[char]] = 1.0

    return encoder_input_data, decoder_input_data, decoder_target_data, encoder_val_input_data, decoder_val_input_data, decoder_val_target_data


In [ ]:
encoder_input_data, decoder_input_data, decoder_target_data, encoder_val_input_data, decoder_val_input_data, decoder_val_target_data =one_hot_encoding(input, target, val_input, val_target, input_tokens, target_tokens)

In [ ]:
encoder_input_data.shape

(58550, 25)

In [ ]:
print(decoder_input_data[0].shape)

(20,)


In [ ]:
def rnn( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder

  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)
  #enc_out = enc_in

  enc_layers = []
  enc_states = []

  for i in range(num_encoders):
        encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        enc_layers.append(encoder)
        enc_out, state_h = encoder(enc_out)
        enc_states.append([state_h])

  # if num_encoders > 1:
  #   encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   enc_layers.append(encoder)
  #   enc_out, state_h = encoder(enc_out)
  #   enc_states.append([state_h2])
  
  # if num_encoders > 2:
  #   encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   enc_layers.append(encoder)
  #   enc_out, state_h = encoder(enc_out)
  #   enc_states.append([state_h3])
  
  
  
  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []

  for i in range(num_decoders):
        decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        dec_layers.append(decoder)
        dec_out, _ = decoder(dec_out, initial_state = enc_states[i])

  
  # if num_decoders > 1 :
  #   decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   dec_layers.append(decoder)
  #   dec_out, _ = decoder(dec_out, initial_state = enc_states[1])
  
  # if num_decoders > 2 :
  #   decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   dec_layers.append(decoder)
  #   dec_out, _ = decoder(dec_out, initial_state = enc_states[2])
  
  
  # Adding Attention:
  decoder_attn=AdditiveAttention()
  decoder_concat=Concatenate()
  cont_vec,attn_wts=decoder_attn([dec_out,enc_out],return_attention_scores=True)
  dec_out= decoder_concat([dec_out,cont_vec])
  
  dec_dense =TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def lstm( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  
  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []

  for i in range(num_encoders):
        encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        enc_layers.append(encoder)
        enc_out, state_h, state_c = encoder(enc_out)
        enc_states.append([state_h, state_c])

  # if num_encoders > 1:
  #   encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   enc_layers.append(encoder)
  #   enc_out, state_h2, state_c2 = encoder(enc_out)
  #   enc_states.append([state_h2, state_c2])
      
  # if num_encoders > 2:
  #   encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   enc_layers.append(encoder)
  #   enc_out, state_h3, state_c3 = encoder(enc_out)
  #   enc_states.append([state_h3, state_c3])
      

  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []

  for i in range(num_decoders):
        decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        dec_layers.append(decoder)
        dec_out, _, _ = decoder(dec_out, initial_state = enc_states[i])


  # if num_decoders > 1:
  #   decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   dec_layers.append(decoder)
  #   dec_out, _, _ = decoder(dec_out, initial_state = enc_states[1])

  # if num_decoders > 2:
  #   decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   dec_layers.append(decoder)
  #   dec_out, _, _ = decoder(dec_out, initial_state = enc_states[2])

  # Adding Attention:
  decoder_attn=AdditiveAttention()
  decoder_concat=Concatenate()
  cont_vec,attn_wts=decoder_attn([dec_out,enc_out],return_attention_scores=True)
  dec_out= decoder_concat([dec_out,cont_vec])
  
  dec_dense = TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def gru(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  
 
  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []

  for i in range(num_encoders):
        encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        enc_layers.append(encoder)
        enc_out, state_h = encoder(enc_out)
        enc_states.append([state_h])
  
  # if num_encoders > 1:
  #   encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   enc_layers.append(encoder)
  #   enc_out, state_h2 = encoder(enc_out)
  #   enc_states.append([state_h2])

  # if num_encoders > 2:
  #   encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   enc_layers.append(encoder)
  #   enc_out, state_h3 = encoder(enc_out)
  #   enc_states.append([state_h3])

  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []

  for i in range(num_decoders):
        decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        dec_layers.append(decoder)
        dec_out, _ = decoder(dec_out, initial_state = enc_states[i])
  
  # if num_decoders > 1:
  #   decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   dec_layers.append(decoder)
  #   dec_out, _ = decoder(dec_out, initial_state = enc_states[1])


  # if num_decoders > 2:
  #   decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   dec_layers.append(decoder)
  #   dec_out, _ = decoder(dec_out, initial_state = enc_states[2])
    
  # Adding Attention:
  decoder_attn=AdditiveAttention()
  decoder_concat=Concatenate()
  cont_vec,attn_wts=decoder_attn([dec_out,enc_out],return_attention_scores=True)
  dec_out= decoder_concat([dec_out,cont_vec])
  
  
  dec_dense = TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def transliteration(config):
  num_encoders=config["num_encoders"]
  cell=config["cell"]
  embed_size=config["embed_size"]
  dropout=config["dropout"]
  num_decoders=config["num_decoders"]
  hidden_layer_size=config["hidden_layer_size"]

  if cell == "RNN":
    model, enc_layers, dec_layers=rnn( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  elif cell == "LSTM":
    model, enc_layers, dec_layers=lstm(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  elif cell == "GRU":
    model, enc_layers, dec_layers=gru(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  


In [ ]:

batch_size = 128        
epochs = 25             
input_embedding_size = 512
hidden_layer_size = 256
num_layers = 3
num_encoder_layers = num_layers
num_decoder_layers = num_layers
dropout = 0.2
cell_type = 'LSTM'

config = {
    "num_encoders" : 3,
    "cell": cell_type,
    "embed_size": 512,
    "dropout": 0.2,
    "num_decoders" : 3,
    "hidden_layer_size":256
}

# TRAIN
model, encoder_layers, decoder_layers = transliteration(config)

# COMPILE
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# FIT
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    shuffle = True,
    # validation_data= ([encoder_val_input_data, decoder_val_input_data], decoder_val_target_data)
)

Epoch 1/25
458/458 [==============================] - 58s 95ms/step - loss: 1.1000 - accuracy: 0.1822
Epoch 2/25
458/458 [==============================] - 43s 93ms/step - loss: 0.4346 - accuracy: 0.3622
Epoch 3/25
458/458 [==============================] - 43s 94ms/step - loss: 0.3190 - accuracy: 0.3930
Epoch 4/25
458/458 [==============================] - 43s 94ms/step - loss: 0.2820 - accuracy: 0.4016
Epoch 5/25
458/458 [==============================] - 43s 94ms/step - loss: 0.2592 - accuracy: 0.4065
Epoch 6/25
458/458 [==============================] - 43s 94ms/step - loss: 0.2424 - accuracy: 0.4097
Epoch 7/25
458/458 [==============================] - 43s 94ms/step - loss: 0.2290 - accuracy: 0.4121
Epoch 8/25
458/458 [==============================] - 43s 94ms/step - loss: 0.2167 - accuracy: 0.4143
Epoch 9/25
458/458 [==============================] - 43s 93ms/step - loss: 0.2061 - accuracy: 0.4160
Epoch 10/25
458/458 [==============================] - 43s 94ms/step - loss: 0.195

In [ ]:
def inferencing(model,num_encoder_layers,num_decoder_layers,encoder_layers,decoder_layers,cell_type, hidden_layer_size):
    
    # ENCODER MODEL RECONSTRUCTION 
    encoder_inputs = model.input[0]  # input_1
    encoder_states = []
    enc_emb = model.layers[2]     # embedding 1
    encoder_outputs = enc_emb(encoder_inputs)

    if cell_type == 'RNN' or cell_type =="GRU":
        for i in range(num_encoder_layers):
            encoder_outputs, state_h_enc = encoder_layers[i](encoder_outputs)
            encoder_states += [state_h_enc] 
    else:
        for i in range(num_encoder_layers):
            encoder_outputs, state_h_enc, state_c_enc = encoder_layers[i](encoder_outputs)
            encoder_states += [state_h_enc, state_c_enc]   

    encoder_model = Model(encoder_inputs, encoder_states + [encoder_outputs])


    # DECODER MODEL RECONSTRUCTION
    input_names = [["input_100","input_101"],["input_102","input_103"],["input_104","input_105"],"input_106"]

    decoder_inputs = model.input[1]       # input_2
    decoder_embedding = model.layers[3]   # embedding 2
    decoder_outputs = decoder_embedding(decoder_inputs)
    decoder_states = []
    decoder_states_inputs = []
    
    if cell_type == 'RNN' or cell_type =="GRU":
        for i in range(num_decoder_layers):
            decoder_states_inputs += [Input(shape=(hidden_layer_size,), name=input_names[i][0])]
        for i in range(num_decoder_layers):
            decoder_outputs, state_h_dec = decoder_layers[i](decoder_outputs, initial_state=decoder_states_inputs[i])
            decoder_states += [state_h_dec]
    else:
        for i in range(num_decoder_layers):
            decoder_states_inputs += [Input(shape=(hidden_layer_size,), name=input_names[i][0]), Input(shape=(hidden_layer_size,), name=input_names[i][1])]
        j = 0
        for i in range(num_decoder_layers):
            decoder_outputs, state_h_dec, state_c_dec = decoder_layers[i](decoder_outputs, initial_state=decoder_states_inputs[i+j:i+j+2])
            decoder_states += [state_h_dec , state_c_dec]
            j += 1

    att_layer = model.layers[4+2*num_encoder_layers]
    attn_input = Input(shape=(max_encoder_seq_length,hidden_layer_size), name=input_names[-1])   

    context_vec, attn_weights = att_layer([decoder_outputs, attn_input], return_attention_scores=True)
    
    concat_layer = model.layers[5+2*num_encoder_layers]
    decoder_outputs = concat_layer([decoder_outputs, context_vec])

    decoder_dense = model.layers[6+2*num_encoder_layers]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs + [attn_input], [decoder_outputs] + decoder_states + [attn_weights])

    return encoder_model, decoder_model

In [ ]:
def decode_sequence(input_seq,encoder_model,decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    attn_input = states_value[-1]
    states_value = states_value[:-1]
    target_seq = np.zeros((1, 1)) 
    target_seq[0, 0] = target_token_index["\t"]
    attn_weights = []
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens = decoder_model.predict([target_seq] + states_value + [attn_input])
        sampled_token_index = np.argmax(output_tokens[0][0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = output_tokens[1:-1]
        attn_weights.append(output_tokens[-1][0][0])
        
    return decoded_sentence, attn_weights

In [ ]:
encoder_model, decoder_model = inferencing(model, num_encoder_layers, num_decoder_layers, encoder_layers, decoder_layers, cell_type, hidden_layer_size)
correct = 0
n = val_target.shape[0]
for i in range(20):
    inputs = encoder_val_input_data[i:i+1]
    output = decode_sequence(inputs,encoder_model, decoder_model)
    print("-")
    print("Input sentence:", val_input[i])
    print("Decoded sentence:", output)

-
Input sentence: amka
Decoded sentence: ('అంకా\n', [array([2.8358714e-04, 1.2620219e-02, 9.0744209e-01, 5.6713074e-02,
       1.0012891e-03, 1.3506875e-04, 5.2776533e-05, 2.4397350e-05,
       2.2611919e-05, 3.6020789e-05, 4.9691953e-05, 5.0368893e-05,
       9.4918978e-05, 1.8554942e-04, 2.7332609e-04, 4.9122795e-04,
       7.9177221e-04, 1.1892486e-03, 1.5822863e-03, 1.9326655e-03,
       2.2665549e-03, 2.6173564e-03, 2.9974445e-03, 3.3915066e-03,
       3.7549513e-03], dtype=float32), array([3.5700730e-06, 1.7434459e-04, 6.2351215e-01, 3.7202084e-01,
       2.8004581e-03, 1.6912275e-04, 5.5696888e-05, 1.7516330e-05,
       6.5574200e-06, 4.5494362e-06, 4.2200018e-06, 4.1030517e-06,
       7.6176002e-06, 1.3762643e-05, 1.9851443e-05, 2.9061837e-05,
       4.3657870e-05, 6.5880376e-05, 8.8417015e-05, 1.0848638e-04,
       1.2750787e-04, 1.4752346e-04, 1.6937748e-04, 1.9224308e-04,
       2.1348194e-04], dtype=float32), array([7.2331147e-07, 3.7247753e-06, 2.3842086e-03, 8.4564507e-01

In [ ]:
def train():
  default_config={
      "cell": "RNN",
      "embed_size":256,
      "dropout":0.2,
      "num_encoders": 1,
      "num_decoders": 1,
      "hidden_layer_size":128,
      "epochs": 1,
      "batch_size": 64
  }

  wandb.init(config=default_config,project=project_name,entity=entity_name)
  config=wandb.config
  wandb.run.name=(
      str(config.cell)
      +"_"+str(config.embed_size)
      +"_"+str(config.dropout)
      +"_"+str(config.num_encoders)
      +"_"+str(config.num_decoders)
      +"_"+str(config.hidden_layer_size)
      +"_"+str(config.epochs)
      +"_"+str(config.batch_size)
  )

  wandb.run.save()

  model=transliteration(config,inp_token_index,tgt_token_index)
  
  model.compile(
      optimizer="adam",
      loss="categorical_crossentropy",
      metrics=["accuracy"]
  )
  earlystopping = EarlyStopping(
          monitor="val_accuracy", min_delta=0.01, patience=5, verbose=2, mode="auto"
      )

  model.fit(
      [e_in,d_in],
      d_t,
      batch_size=config.batch_size,
      epochs=config.epochs,
      callbacks=[earlystopping,WandbCallback()]
  )

  wandb.finish()
  

In [ ]:
def sweeper(project_name,entity_name)
  sweep_configuration={
      "method": "bayes",
      "metric": {"name": "val_accuracy", "goal": "maximize"},
      "parameters":{
          "cell":{"values":["RNN","GRU","LSTM"]},
          "embed_size":{"values":[16,32,64,256]},
          "hidden_layer_size":{"values":[16,32,64,256]},
          "num_encoders":{"values":[1,2,3]},
          "num_decoders":{"values":[1,2,3]},
          "dropout":{"values":[0.2,0.3]},
          "epochs":{"values":[5,10,15,20]},
          "batch_size":{"values":[32,64]}
      }
  }

  sweep_id=wandb.sweep(sweep_configuration,project=project_name,entity=entity_name)
  wandb.agent(sweep_id,train)


In [ ]:
# Running the sweeps for hyperparameter tuning
sweeper(project_name,entity_name)